In [ ]:
import numpy as np
import scipy.stats as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import minmax_scale, scale, MinMaxScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
from wgan.models_cat import Generator, Critic
from wgan.training import WGAN
from wgan.dataloaders import TabularDataset

import torch
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, brier_score_loss

In [ ]:
from torch.utils.data import Dataset, DataLoader
from imbalanced_sampler.sampler import ImbalancedDatasetSampler

In [ ]:
def make_correlation_matrix(no_var):
    corr = np.zeros([no_var,no_var])
    corr_temp = np.random.uniform(-1,1,size=[(no_var-1)*2])
    corr[np.triu_indices(no_var, 1)] = corr_temp
    corr + corr.T + np.eye(no_var)
    return corr


def create_continuous_data(N, pos_ratio=0, noise_ratio=0, no_var=10, cov=None, random_state=None):
    if random_state is not None: np.random.seed(random_state)
    # Group indicator
    #group = sp.binom.rvs(p=0.25, n=1, size=N    
    N_neg = int(N*(1-pos_ratio))
    N_pos = N-N_neg
    y = np.concatenate([np.zeros(N_neg), np.ones(N_pos)])
    
    mean = np.random.uniform(size=no_var)
    mean0 = np.random.normal(loc=mean,scale=0.5)
    mean1 = np.random.normal(loc=mean,scale=0.5)
    
    if cov is None: 
        cov0 = sp.invwishart.rvs(df=no_var*2, scale=np.eye(no_var))
        cov1 = sp.invwishart.rvs(df=no_var*2, scale=np.eye(no_var))

    # Noise are variables with same distribution in majority and minority class
    if noise_ratio != 0:  
        no_noise = int(noise_ratio*no_var)
        no_var = no_var - no_noise
        X_noise = sp.multivariate_normal.rvs(mean=mean0[no_var:], cov=cov0[no_var:,no_var:], size=N).reshape([N,-1])

    X1 = sp.multivariate_normal.rvs(mean=mean1[0:no_var], cov= cov1[:no_var,:no_var], size=N_pos)
    X0 = sp.multivariate_normal.rvs(mean=mean0[0:no_var], cov= cov0[:no_var,:no_var], size=N_neg)
    X = np.vstack([X0,X1])
    X = np.hstack([X, X_noise])
    
    return {"X":X, "y":y,"mean0":mean0,"mean1":mean1, "cov0":cov0, "cov1":cov1}

def create_dataset(n_samples=1000, n_features=2, n_classes=3, weights=(0.01, 0.01, 0.98),
                   class_sep=0.8, n_clusters=1, random_state=0):
    return make_classification(n_samples=n_samples,
                               n_informative=2, n_redundant=0, n_repeated=0,
                               n_classes=n_classes, n_features = n_features,
                               n_clusters_per_class=n_clusters,
                               weights=list(weights),
                               class_sep=class_sep, random_state=random_state)

## Artifical Data Generation

In [ ]:
modus = 'minority' #'full

In [ ]:
no_cont = 4
no_cat = 4
no_vars = no_cont + no_cat
N= 50000

# Create single dataset to avoid random effects
# Only works for all informative features
X_full,y = make_classification(n_samples=N, weights=[0.9,0.1], n_clusters_per_class=1,
                              n_features=no_vars, 
                              n_informative=no_vars, 
                              n_redundant=0, n_repeated=0,
                             random_state=123)

In [ ]:
ct = ColumnTransformer([
    ("scaler", MinMaxScaler(), slice(no_cont)),
    ("discretizer", KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="quantile"),
     slice(no_cont,))
], remainder="drop")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y, 
                                                    stratify=y, test_size=0.5, random_state=123)


X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

X_train_majority = X_train[y_train==0,:]
X_train_minority = X_train[y_train==1,:]

y_train_bin = y_train[:]
y_test_bin = y_test[:]
y_temp = np.zeros([len(y_train),2])
y_temp[y_train==0,0] = 1
y_temp[y_train==1,1] = 1
y_train = y_temp

In [ ]:
cat_levels = [len(np.unique(X_train[:,i])) for i in range(no_cont, no_vars)]

#mean_minority = np.mean(X_minority, axis=0)
#sd_minority = np.std(X_minority, axis=0)
#X_minority = (X_minority-mean_minority)/sd_minority

if modus == 'minority':
    data_train = TabularDataset(X = X_train_minority[:,:no_cont],
                             X_cat = X_train_minority[:,no_cont:],
                             y = y_train[np.argmax(y_train, axis=1),:],
                                cat_levels = cat_levels
                               )
elif modus == 'full':
    data_train = TabularDataset(X = X_train[:,:no_cont],
                             X_cat=X_train[:,no_cont:], 
                             y=y_train)
    data_test = TabularDataset(X = X_test[:,:no_cont],
                             X_cat=X_test[:,no_cont:], 
                             y=y_test)
else:
    stop("Check modus. Must be one of ['minority, 'full]")

In [ ]:
emb_size = 5
cat_inputs = list(zip(cat_levels, [emb_size] * len(cat_levels)))
cont_inputs = no_cont

In [ ]:
#cat_levels=0
#cat_inputs=0
#data_train = TabularDataset(X = X_train[:,:no_cont],
#                         X_cat=None, 
#                         y=y_train)

In [ ]:
generator = Generator(latent_dim=10, lin_layer_sizes=[64,128], 
                      output_dim=cont_inputs, cat_output_dim=cat_levels, aux_dim=0) #[10,10]

critic = Critic(input_size=cont_inputs, lin_layer_sizes=[64,128,128], 
                              cat_input_sizes=cat_inputs, 
                aux_input_size=0) #[(10,1),(10,1)]

print(generator)
print(critic)

In [ ]:
batch_size = 128
train_loader = DataLoader(data_train, batch_size = batch_size, shuffle=True)
#test_loader = DataLoader(data_test, batch_size = batch_size, shuffle=False)

# Balanced sampling through inverse propensiImbalancedDatasetSampler(labels = list(y_train), num_samples=batch_size)ty
#data_loader = DataLoader(dataset, batch_size = batch_size, 
#                     sampler = sampler)

In [ ]:
# Initialize optimizers
lr_G = 5e-5
lr_D = 5e-5
betas = (.9, .99)
G_optimizer = optim.Adam(generator.parameters(), lr=lr_G, betas=betas)
C_optimizer = optim.Adam(critic.parameters(), lr=lr_D, betas=betas)

In [ ]:
trainer = WGAN(generator, critic, G_optimizer, C_optimizer, print_every=1000,
                  use_cuda=torch.cuda.is_available())

In [ ]:
trainer.gp_weight = 10.

In [ ]:
trainer.train(train_loader, 10000)

In [ ]:
generator.training_iterations

In [ ]:
desc = f"cont_cat_n{N//1000}_k{no_vars}_{modus}"
torch.save(generator.state_dict(), f"../models/wgan_generator_{desc}_{generator.training_iterations}")
torch.save(critic.state_dict(), f"../models/wgan_critic_{desc}_{generator.training_iterations}")

In [ ]:
file_name = "cont_cat_n50_k8_minority_95191"
generator.load_state_dict(torch.load(f"../models/wgan_generator_{file_name}"))
critic.load_state_dict(torch.load(f"../models/wgan_critic_{file_name}"))

## Visual test

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
combinations = [(x,y) for x in range(no_vars) for y in range(no_vars) if y>x]

In [ ]:
fig, axes = plt.subplots(nrows=no_vars, ncols=no_vars, sharex=True, sharey=True, squeeze=True,figsize=(10,10))
for y in axes:
    for x in y:
        x.set_xticklabels([])
        x.set_yticklabels([])

for i,j in combinations:
    sns.kdeplot(X_majority[:,i], X_majority[:,j], alpha=0.5, cmap="Blues", ax=axes[(j,i)])
    sns.kdeplot(X_minority[:,i], X_minority[:,j], alpha=0.5, cmap="Greens", ax=axes[(j,i)])
fig.savefig(f'../img/cat_sample_tr_iter_{trainer.G.training_iterations}.png',format='png', dpi=100)
    #fig.show()

In [ ]:
epochs = 90

for _ in range(30):
    trainer.train(data_loader, epochs)
    
    
    if modus == 'full':
        fake_minority = generator(*generator.sample_latent(num_samples= 1000, class_index=1)).data.numpy()
        fake_majority = generator(*generator.sample_latent(num_samples= 1000, class_index=0)).data.numpy()
    elif modus == 'minority':
        fake_minority = generator(generator.sample_latent(num_samples= 1000)).data.numpy()
        
    fig, axes = plt.subplots(nrows=no_vars, ncols=no_vars, sharex=True, squeeze=True,figsize=(10,10))
    for y in axes:
        for x in y:
            x.set_xticklabels([])
            x.set_yticklabels([])
    
    for i in range(no_vars):
        sns.kdeplot(X_minority[:,i], alpha=0.5, shade=True, color="blue", ax=axes[(i,i)])
        sns.kdeplot(fake_minority[:,i], alpha=0.5, shade=True, color="green", ax=axes[(i,i)])
    
    for i,j in combinations:
        axes[(i,j)].set_ylim(0,1)
        # majority (upper right)
        if modus == 'full':
            sns.kdeplot(X_majority[0:1000,i], X_majority[0:1000,j], alpha=0.5, cmap="Blues", ax=axes[(i,j)])
            sns.kdeplot(fake_majority[:,i], fake_majority[:,j], alpha=0.5, cmap="Greens", ax=axes[(i,j)], )
        
        # minority (lower left)
        sns.kdeplot(X_minority[:,i], X_minority[:,j], alpha=0.5, cmap="Blues", ax=axes[(j,i)])
        sns.kdeplot(fake_minority[:,i], fake_minority[:,j], alpha=0.5, cmap="Greens", ax=axes[(j,i)])
        
    fig.savefig(f'../img/cont_sample_tr_iter_{trainer.G.training_iterations}.png',format='png', dpi=200)
        #fig.show()

## Distribution summary statistics

In [ ]:
#fake_minority = generator(generator.sample_latent(num_samples= X_train_minority.shape[0], class_index=None))
fake_minority = generator.sample_data(10000).numpy()
minority = pd.DataFrame(X_train_minority)

In [ ]:
for i in range(minority.shape[1]):
    plt.hist(X_train_minority[:,i], alpha=0.3, density=True)
    plt.hist(fake_minority[:,i], alpha=0.3, density=True, color='red')
    plt.show()

In [ ]:
print(np.quantile(X_train_minority, q=np.arange(0,1,0.1), axis=0))
print(np.quantile(fake_minority[0].data.numpy(), q=np.arange(0,1,0.1), axis=0))

In [ ]:
print(np.cov(X_minority, rowvar=False) - np.cov(fake_minority,rowvar=False))


## Discriminator test

In [ ]:
sample_size = X_train_minority.shape[0]

In [ ]:
#fake = generator(*generator.sample_latent(num_samples= sample_size, class_index=1)).data.numpy()
fake = generator.sample_data(num_samples= sample_size)

In [ ]:
X_fakereal = np.vstack([X_train_minority, 
                        fake])
y_fakereal = np.concatenate([np.zeros(X_train_minority.shape[0]), 
                        np.ones(fake.shape[0])]).flatten()

In [ ]:
clf = RandomForestClassifier(n_estimators=50, min_samples_leaf=10, n_jobs=10)
model_fakereal = clf.fit(X_fakereal, y_fakereal)

In [ ]:
pred_fakereal = model_fakereal.predict_proba(X_fakereal)[:,1]
print(accuracy_score(y_fakereal, pred_fakereal>0.5))
roc_auc_score(y_fakereal, pred_fakereal)


# Predictive performance testing

In [ ]:
def test_auc(model_library, X, y_true):
    auc = {}
    for model in model_library.keys():
        pred = model_library[model].predict_proba(X)[:,1]
        auc[model] = roc_auc_score(y_true, pred)
    return auc

## Predictive test

In [ ]:
minority_samples = X_minority.shape[0]
majority_samples = X_majority.shape[0]

fake_minority = generator(*generator.sample_latent(num_samples= minority_samples, class_index=1)).data.numpy()
fake_majority = generator(*generator.sample_latent(num_samples= majority_samples, class_index=0)).data.numpy()

X_synthetic = np.vstack([fake_majority, 
                         fake_minority])
y_synthetic = np.concatenate([np.zeros(majority_samples), 
                              np.ones(minority_samples)]).flatten()

In [ ]:
clf_org = DecisionTreeClassifier(max_depth=10) #LogisticRegression(solver='saga') 
clf_fake = DecisionTreeClassifier(max_depth=10) #LogisticRegression(solver='saga')

predictive = {}
predictive["real"] = clf_org.fit(X=X_train, y=y_train_bin)
predictive["synthetic"] = clf_fake.fit(X=X_synthetic, y=y_synthetic)

test_auc(predictive, X_test, y_test)

## Upsampling performance

In [ ]:
upsampling_ratio = 4
sample_size = int(X_train_minority.shape[0] * upsampling_ratio)

In [ ]:
performance = {'train':{"original":[],"GANbalanced":[]}, 'test':{"original":[],"GANbalanced":[]}}
for _ in range(10):
    #X_fake = generator(*generator.sample_latent(num_samples= sample_size, class_index=1)).data.numpy()
    X_fake = generator.sample_data(sample_size).numpy()
    y_fake = np.ones(shape=[sample_size])

    X_up = np.vstack([X_train,X_fake])
    y_up = np.hstack([y_train_bin,y_fake])

    clf_org = DecisionTreeClassifier(min_weight_fraction_leaf=0.01)
    clf_fake = DecisionTreeClassifier(min_weight_fraction_leaf=0.01)

    upsampling = {}
    upsampling["original"] =  clf_org.fit(X=X_train, y=y_train_bin)
    upsampling["GANbalanced"] = clf_fake.fit(X=X_up, y=y_up)
    
    performance_temp_train = test_auc(upsampling, X_train, y_train_bin)
    performance_temp_test = test_auc(upsampling, X_test, y_test_bin)
    
    for model in performance_temp_test:
        performance['train'][model].append(performance_temp_train[model])
        performance['test'][model].append(performance_temp_test[model])
    

In [ ]:
print(pd.DataFrame(performance['train']).mean())
print(pd.DataFrame(performance['test']).mean())


print(pd.DataFrame(performance['test']).std())

In [ ]:
def plot_decision_function(X, y, clf, ax):
    plot_step = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.4)
    ax.scatter(X[:, 0], X[:, 1], alpha=0.8, c=y, edgecolor='k')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
plot_decision_function(X_train, y_train, upsampling["original"], ax1)
plot_decision_function(X_up, y_up, upsampling["GANbalanced"], ax2)

fig.tight_layout()

## Effect of dimensionality on SMOTE

With increasing dimensionality, we expect SMOTE's underlying nearest neighbor approach to fail to capture relevant neighborhoods. We measure SMOTE performance in terms of RF being able to differentiate between real and synthetic data.

In [ ]:
from imblearn.over_sampling import SMOTENC, SMOTE
from imblearn.under_sampling import TomekLinks

In [ ]:
n_features = 320
# Create single dataset to avoid random effects
# Only works for all informative features
X_full,y = make_classification(n_samples=10000, weights=[0.9,0.1], n_clusters_per_class=1,
                              n_features=n_features, 
                              n_informative=n_features, 
                              n_redundant=0, n_repeated=0,
                             random_state=123)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y, 
                                                    stratify=y, test_size=0.5, random_state=123)

In [ ]:
## Oversampling test
# Drop variables until desired dimensionality
no_var_list = [5,10,20,40,80,160,320]
auc_no_vars = []

for k in no_var_list: #
#     X_full,y = make_classification(n_samples=1000, weights=[0.9,0.1], n_clusters_per_class=1,
#                               n_features=k, 
#                               n_informative=k, 
#                               n_redundant=0, n_repeated=0,
#                              random_state=123)
#     X_train_sample, X_test_sample, y_train, y_test = train_test_split(X_full, y, 
#                                                     stratify=y, test_size=0.5, random_state=123)
    X_train_sample = X_train[:,0:k]
    X_test_sample = X_test[:,0:k]
    
    # Sample synthetic SMOTE data
    smote = SMOTE(sampling_strategy = {1:np.sum(y)*1}, k_neighbors=10,
                  random_state=123, n_jobs=20)
    X_smote, y_smote =  smote.fit_resample(X_train_sample,y_train)
    
    # Supplement original data
    model_library = {
        "original":RandomForestClassifier(n_estimators=100, min_samples_leaf=50, n_jobs=20),
        "smote":RandomForestClassifier(n_estimators=100, min_samples_leaf=50, n_jobs=20)
    }

    model_library["original"] = model_library['original'].fit(X=X_train_sample, y=y_train)
    model_library["smote"] = model_library['smote'].fit(X=X_smote, y=y_smote)
    
    temp = test_auc(model_library, X_test_sample, y_test)
    auc_no_vars.append(temp)
    

In [ ]:
auc_no_vars

In [ ]:
def fake_real_score(X_real, X_fake, random_state=None):
    X_fakereal = np.vstack([X_real, X_fake])
    y_fakereal = np.concatenate([np.zeros(X_real.shape[0]), 
                                 np.ones( X_fake.shape[0])]).flatten()
    
    X_fakereal_train, X_fakereal_test, y_fakereal_train, y_fakereal_test =\
        train_test_split(X_fakereal, y_fakereal, test_size=0.5, random_state=random_state)
    clf = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=0.05, n_jobs=20, random_state=random_state)
    model_fakereal = clf.fit(X_fakereal_train, y_fakereal_train)

    pred_fakereal = model_fakereal.predict_proba(X_fakereal_test)[:,1]
    return roc_auc_score(y_fakereal_test, pred_fakereal)

In [ ]:
## Discriminator test
# Drop variables until desired dimensionality
auc = {}

for k in [5,10,20,40,80,160,320]: #
    X = X_full[:,0:k]
    # Sample synthetic SMOTE data
    smote = SMOTE(sampling_strategy = {1:np.sum(y)*2}, k_neighbors=5,
                  random_state=123, n_jobs=20)
    X_smote, y_smote = smote.fit_sample(X,y)
   
    auc[k] = fake_real_score(X[y==1],X_smote, random_state=123)
    
print(auc)

In [ ]:
X.shape

In [ ]:
## Discriminator test
# Drop variables until desired dimensionality
auc = {}

for k in [5,10,20,40,80,160,320]: #
    X = X_full[:,0:k]
    # Sample synthetic GAN data
    # TODO: Train GAN generator
    X_gan = generator.sample_data(np.sum(y)*2)
   
    auc[k] = fake_real_score(X_gan,X_smote, random_state=123)
    
print(auc)

In [ ]:
plt.plot(auc.keys(), auc.values())
plt.xlabel("No. of variables (10,000 minority observations )")
plt.ylabel("Discriminator AUC (SMOTE)")
plt.savefig("../img/SMOTE_performance_over_variables_10k_minority.png", format='png',dpi=200)
#plt.show()